In [1]:
# !pip install sentencepiece
# !pip install pyaspeller

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from transformers import T5ForConditionalGeneration, T5Tokenizer
import torch
import pandas as pd
from tqdm import tqdm
from pyaspeller import YandexSpeller

In [3]:
import math

def rescore(text: str, tokenizer: T5Tokenizer,
            model: T5ForConditionalGeneration) -> str:
    
    if isinstance(text, float) and math.isnan(text): # if an input text is empty, then we return an empty text too
        return ''
    elif len(text) == 0:  # if an input text is empty, then we return an empty text too
        return ''
    ru_letters = set('аоуыэяеёюибвгдйжзклмнпрстфхцчшщьъ')
    punct = set('.,:/\\?!()[]{};"\'-')
    x = tokenizer(text, return_tensors='pt', padding=True).to(model.device)
    max_size = int(x.input_ids.shape[1] * 1.5 + 10)
    min_size = 3
    if x.input_ids.shape[1] <= min_size:
        return text  # we don't rescore a very short text
    out = model.generate(**x, do_sample=False, num_beams=5,
                         max_length=max_size, min_length=min_size)
    res = tokenizer.decode(out[0], skip_special_tokens=True).lower().strip()
    res = ' '.join(res.split())
    postprocessed = ''
    for cur in res:
        if cur.isspace() or (cur in punct):
            postprocessed += ' '
        elif cur in ru_letters:
            postprocessed += cur
    return (' '.join(postprocessed.strip().split())).replace('ё', 'е')

In [30]:
speller = YandexSpeller()

def rescore_ya_speller(text):
    if isinstance(text, float) and math.isnan(text): # if an input text is empty, then we return an empty text too
        return ''
    elif len(text) == 0:  # if an input text is empty, then we return an empty text too
        return ''
    
    fixed = speller.spelled(text)
    return fixed    

rescore_ya_speller("мишинное обеченее клоссная весчь")

'машинное обучение классная весчь'

# Load Transcriptions

In [36]:
# test_df = pd.read_csv('test_df_denoisy.csv')
# test_df = pd.read_csv('test_df_finetune.csv')
test_df = pd.read_csv('test_df_default_asr.csv')

test_df.head(1)
test_df

,transcription,gt
0,к сожаление эти предложения не насвои ображани...,"К сожалению, эти предложения не нашли отражени..."
1,если не будет возражений я буду считать что ас...,"Если не будет возражений, я буду считать, что ..."
2,на вошаркен милый вород,Новошахтинск — милый город
3,сё рану разозвась что числе теряющихсяот поещё...,"Мы особенно рады отметить, что число скрывающи..."
4,контролер,Контроллер
...,...,...
9625,мы должны сообща вешать задачи в области остра...,Мы должны сообща решать задачи в области охран...
9626,требуваммое большинство в две третьих,Требуемое большинство в две трети.
9627,председатель вновь занимает место председателя,Председатель вновь занимает место Председателя.
9628,давайте же по случаю ее кодобщины вновь подтве...,Давайте же по случаю ее годовщины вновь подтве...


# Rescore by YaSpeller

In [37]:
test_rescored_results = []

for i in tqdm(test_df["transcription"]):
    rescored = rescore_ya_speller(i) 
    test_rescored_results.append(rescored)
    
test_df["rescored_transcription"] = test_rescored_results

100%|██████████| 9630/9630 [26:34<00:00,  6.04it/s]  


In [38]:
# test_df.to_csv('test_df_denoisy_rescored_yaspeller.csv', index=False)
# test_df.to_csv('test_df_finetune_rescored_yaspeller.csv', index=False)
test_df.to_csv('test_df_default_asr_rescored_yaspeller.csv', index=False)

# Rescore by T5

In [13]:
model_name = 'bond005/ruT5-ASR'

tokenizer_for_rescoring = T5Tokenizer.from_pretrained(model_name)
model_for_rescoring = T5ForConditionalGeneration.from_pretrained(model_name)

if torch.cuda.is_available():
    model_for_rescoring = model_for_rescoring.cuda()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [34]:
test_rescored_results = []

for i in tqdm(test_df["transcription"]):
    rescored = rescore(i, tokenizer_for_rescoring, model_for_rescoring)
    test_rescored_results.append(rescored)
    
test_df["rescored_transcription"] = test_rescored_results

100%|██████████| 9630/9630 [33:10<00:00,  4.84it/s]


In [35]:
test_df

,transcription,gt,rescored_transcription
0,сожаление эти предложения наносли отражения в ...,"К сожалению, эти предложения не нашли отражени...",к сожалению эти предложения наносили отражение...
1,если не будет возражений я буду считать что ас...,"Если не будет возражений, я буду считать, что ...",если не будет возражений я буду считать что ас...
2,навошафкин вскмилый город,Новошахтинск — милый город,на ворошилова всколыхнулся город
3,на особенно рады ознуть что числы ешесят поесе...,"Мы особенно рады отметить, что число скрывающи...",но особенно рады были узнать что числящийся по...
4,контролер,Контроллер,контролер
...,...,...,...
9625,мы должны сообща огешать задачи в области охтр...,Мы должны сообща решать задачи в области охран...,мы должны сообща решать задачи в области охран...
9626,требуемое большинство в две третьих,Требуемое большинство в две трети.,требуемое большинство в две третьих
9627,председатель вновь занимает место председателя,Председатель вновь занимает место Председателя.,председатель вновь занимает место председателя
9628,давайте же по случаю ее боговщины вновь подтве...,Давайте же по случаю ее годовщины вновь подтве...,давайте же по случаю ее годовщины вновь подтве...


In [36]:
# train_df.to_csv('train_df_rescored.csv', index=False)
test_df.to_csv('test_df_finetune_rescored.csv', index=False)